# Tutorial

The CausalInflation package generates semidefinite programming relaxions of inflations of causal scenarios. The implementation follows an object-oriented design. There are three main steps:

* Encode the causal scenario and desired inflation as an instance of `InflationProblem`.
* Generate the semidefinite programming relaxation of the desired inflation through `InflationSDP`.
* Export the relaxation to a file and/or solve the problem.

We will now go through the steps above in more detail. This tutorial is available on the documentation [webpage](https://ecboghiu.github.io/inflation/) and it can also be downloaded as a ready-to-run Jupyter Notebook from the [GitHub repository](https://github.com/ecboghiu/inflation).

We start by importing everything that we will need:

In [1]:
from causalinflation import InflationProblem, InflationSDP

## Encoding the inflation problem 

The first object of interest is `InflationProblem`. When instantiating it, we pass all the relevant information about the causal scenario and the type of inflation we want to perform. Currently, only quantum inflation is implemented.

### Basics of causal diagrams

Causal relationships are encoded through a Bayesian Directed-Acyclic-Graph, or "DAG", for short. The nodes of the graph represent random variables, which can be either observable (visible) or unobservable (hidden). Directed arrows encode causal influence between nodes. Arrows point from parent nodes to the nodes causally influences by the parent, called children nodes. The acyclicity property avoids the presence of causal loops. 

For applications to physics, another class of random variables is often considered called "settings". These correspond to observable variables on whose outcome we condition in order to obtain the observed data. For example, this might correspond to measuring one property of a system versus another in an experiment. Furthermore, in physics we also have sources generating quantum correlations. These are modeled through the presence of quantum unobservable variables, which represent quantum system. Arrows going into a quantum node represent controlling the corresponding quantum system. Arrows going out of the quantum node represent the children of the quantum node generating statistics by measuring the quantum system.

### The bilocal scenario and an inflation

As an example, let us consider the quantum ["bilocal scenario"](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.85.032119), which corresponds to correlations achievable by measuring quantum particles that never interacted in the past. This is a simple scenario where [entanglement swapping](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.71.4287) can be performed, which is at the heart of the [quantum teleportation protocol](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.70.1895). The corresponding DAG is:

<img src="./figures/bilocal_1.PNG" alt="drawing" width="400"/>

We have three observed random variables, $A$, $B$ and $C$, two hidden quantum variables $\rho_{AB}$ and $\rho_{BC}$ denoting the particles and three setting variables, $X$, $Y$ and $Z$. We assume all observed variables have cardinality 2, i.e., $a,b,c,x,y,z \in \{0, 1\}$, where lower case denotes specific values the random variables (in upper case) can take. In this causal scenario, correlations are generated by measuring the quantum systems conditioned on the values of the settings. These correlations are encoded in the joint probability distribution $p(a,b,c|x,y,z)$. Notice that the probability distribution is not conditioned on $\rho_{AB}, \rho_{BC}$ as they are hidden variables.

One of the fundamental questions to ask if whether a given a probability distribution $p(a,b,c|x,y,z)$ is compatible with a given causal scenario. The challenge in answering this question comes from the assumption of statistical independence of the quantum sources $\rho_{AB}$ and $\rho_{BC}$. The quantum inflation technique is one technique that can give an answer to this question by relaxing the independence constraint. Let us consider an inflation of the bilocal scenario where we take two copies of each source, as in the following figure:

<img src="./figures/bilocal_inflated.PNG" alt="drawing" width="400"/>

Compatibility of the probability distribution with the inflated graph can be relaxed to a semidefinite program (SDP) using the NPA hierarcy. At the level of the SDP, the independence of the sources is relaxed to a collection of symmetries that can be applied to the SDP. For more details, see [Physical Review X 11.2 (2021): 021043](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.11.021043).

### Creating an instance of `InflationProblem`

We encode information about the causal scenario as an instance of `InflationProblem` as follows:

In [2]:
InfProb = InflationProblem( dag={"h1": ["v1", "v2"], "h2": ["v2", "v3"]},
                            outcomes_per_party=[2, 2, 2],
                            settings_per_party=[2, 2, 2],
                            inflation_level_per_source=[2, 2])

The DAG is written as a dictionary where the keys are parent nodes, and the key value denotes all the parent's children as a list of nodes. The first character of the node name denotes whether the node is visible or hidden. In `outcomes_per_party` and `settings_per_party` we specify the cardinalities of visible nodes. The paremeter `inflation_level_per_source` determines how many copies of the quantum sources we consider in the inflated graph.

## Generating the SDP relaxation

The next object of interest is `InflationSDP`. It is an object that takes as input an instance of `InflationProblem`:

In [3]:
from causalinflation import InflationSDP

InfSDP = InflationSDP(InfProb)

The first main method, `generate_relaxation()` is that which generates the SDP relaxation of the chosen inflation:

In [4]:
InfSDP.generate_relaxation(column_specification='npa1')

In the above example, we have chosen "NPA hierarchy level 1". For the meaning of these levels, see [New Journal of Physics 10.7 (2008): 073013](https://iopscience.iop.org/article/10.1088/1367-2630/10/7/073013/meta). For other hierarchies that we support, see the documentation of `generate_relaxation()`. What is important to know is that the higher the hierarchy level, the tighter are the 


 There are three steps to generate the relaxation:

Instantiate the SdpRelaxation object.
Get the relaxation.
Write the relaxation to a file or solve the problem.
The second step is the most time consuming, often running for hours as the number of variables increases. Once the solution is obtained, it can be studied further with some helper functions.

To instantiate the SdpRelaxation object, you need to specify the variables. You can use any SymPy symbolic variable, as long as the adjoint operator is well-defined. The library also has helper functions to generate commutative or noncommutative variables or operators.

Getting the relaxation requires at least the level of relaxation, and the matching method, SdpRelaxation.get_relaxation, will generate the moment matrix. Additional elements of the problem, such as the objective function, inequalities, equalities, and constraints on the moments.

The last step in is to either solve or export the relaxation. The function solve_sdp or the class method SdpRelaxation.solve autodetects the possible solvers: SDPA, MOSEK, and CVXOPT. Alternatively, the method write_to_file exports the file to sparse SDPA format, which can be solved externally on a supercomputer, in MATLAB, or by any other means that accepts this input format.